In [ ]:
%pip install pandas numpy regex textblob transformers torch scikit-learn tensorflow setuptools


In [18]:
import pandas as pd
import numpy as np
import regex as re
import string

In [8]:
train_csv = pd.read_csv('Data/train.csv')

In [10]:
train_csv.head()

,tweet_id,user_handle,timestamp,tweet_text,candidate,party,retweets,likes,sentiment
0,1,@user123,2024-11-03 08:45:00,Excited to see Kamala Harris leading the Democ...,Kamala Harris,Democratic Party,120,450,positive
1,2,@politicsFan,2024-11-03 09:15:23,Donald Trump's policies are the best for our e...,Donald Trump,Republican Party,85,300,positive
2,3,@greenAdvocate,2024-11-03 10:05:45,Jill Stein's environmental plans are exactly w...,Jill Stein,Green Party,60,200,positive
3,4,@indieVoice,2024-11-03 11:20:10,Robert Kennedy offers a fresh perspective outs...,Robert Kennedy,Independent,40,150,neutral
4,5,@libertyLover,2024-11-03 12:35:55,Chase Oliver's libertarian stance promotes tru...,Chase Oliver,Libertarian Party,30,120,positive


Data Preprocessing

In [16]:
# Remove HMTL Tages, lowercase, remove @mentions
clean = re.compile('<.*?>')
train_csv['tweet_text'] = train_csv['tweet_text'].str.lower()
train_csv['tweet_text'] = train_csv['tweet_text'].apply(lambda x: re.sub(clean, '',x) )
train_csv['tweet_text'] = train_csv['tweet_text'].str.replace(r'@\S+', '', regex=True)


In [21]:
#remove Puncuations
train_csv['tweet_text'] = train_csv['tweet_text'].apply(lambda x : x.translate(str.maketrans('', '',string.punctuation)))

In [32]:
# spelling corrections
from textblob import TextBlob
train_csv['tweet_text'] = train_csv['tweet_text'].apply(lambda x: TextBlob(x).correct().string)

In [38]:
import random
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity

In [40]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [98]:
text = train_csv['tweet_text'][0]

encoded_input = tokenizer(text,return_tensors='pt')

In [103]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

In [108]:
output['pooler_output'].shape

torch.Size([1, 768])

In [44]:
# Generate embeddings using BERT model
with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
    word_embeddings = outputs.last_hidden_state  # This contains the embeddings

# Output the shape of word embeddings
print(f'Shape of Word Embeddings: {word_embeddings.shape}')

Shape of Word Embeddings: torch.Size([1, 11, 768])


In [46]:
# Decode the token IDs back to text
decoded_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
#print decoded text
print(f'Decoded Text: {decoded_text}')
# Tokenize the text again for reference
tokenized_text = tokenizer.tokenize(decoded_text)
#print tokenized text
print(f'tokenized Text: {tokenized_text}')
# Encode the text
encoded_text = tokenizer.encode(text, return_tensors='pt')  # Returns a tensor
# Print encoded text
print(f'Encoded Text: {encoded_text}')

Decoded Text: excited to see mala harris leading the democratic charge
tokenized Text: ['excited', 'to', 'see', 'mala', 'harris', 'leading', 'the', 'democratic', 'charge']
Encoded Text: tensor([[  101,  7568,  2000,  2156, 28935,  5671,  2877,  1996,  3537,  3715,
           102]])


In [50]:
word_embeddings

tensor([[[-0.1750,  0.0534, -0.1162,  ..., -0.0963,  0.2406,  0.3589],
         [ 0.6159, -0.6472, -0.3491,  ..., -0.1919,  0.4275,  0.5265],
         [ 0.2682, -0.4770,  0.1700,  ..., -0.3569,  0.2511,  0.0313],
         ...,
         [-0.4254, -0.6637, -0.1005,  ..., -0.3504, -0.3146,  0.5374],
         [-0.5015, -0.9129, -0.1265,  ...,  0.1249,  0.0892, -0.3274],
         [ 0.6415,  0.0750, -0.3870,  ...,  0.1416, -0.6869, -0.0764]]])

In [51]:
def word_embeddings_BERT(text):
    text = train_csv['tweet_text'][0]

    encoding = tokenizer.batch_encode_plus( [text],# List of input texts
        padding=True,              # Pad to the maximum sequence length
        truncation=True,           # Truncate to the maximum sequence length if necessary
        return_tensors='pt',      # Return PyTorch tensors
        add_special_tokens=True    # Add special tokens CLS and SEP
    )

    input_ids = encoding['input_ids']  # Token IDs
    # print input IDs
    # print(f'Input ID: {input_ids}')
    attention_mask = encoding['attention_mask']  # Attention mask
    # print attention mask
    # print(f'Attention mask: {attention_mask}')
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        word_embeddings = outputs.last_hidden_state  # This contains the embeddings

    return word_embeddings

In [61]:
word_embeddings_BERT(train_csv['tweet_text'][0]).shape

torch.Size([1, 11, 768])

In [62]:
train_csv['tweet_text_embeded'] = train_csv['tweet_text'].apply(lambda x: word_embeddings_BERT(x)[0])

In [64]:
train_csv.to_csv('Data/train_embeded.csv', index=False)

In [65]:
def extract_cls_or_mean(embedding, method="cls"):
    """
    Extract sentence representation using CLS token or mean pooling.
    """
    # Remove batch dimension: Shape [1, sequence_length, 768] -> [sequence_length, 768]
    embedding = embedding.squeeze(0)
    
    if method == "cls":
        return embedding[0]  # CLS token embedding
    elif method == "mean":
        return embedding.mean(dim=0)  # Mean pooling
    else:
        raise ValueError("Unsupported pooling method. Choose 'cls' or 'mean'.")
    
train_csv["sentence_embedding"] = train_csv["tweet_text_embeded"].apply(lambda x: extract_cls_or_mean(x, method="cls"))

In [68]:
train_csv['sentence_embedding'][0].shape

torch.Size([768])

In [82]:
train_csv['sentiment'].value_counts()

sentiment
positive      328
neutral       126
negative       45
positive        1
Name: count, dtype: int64

In [72]:
label_mapping = {"negative": 0, "neutral": 1, "positive": 2}
train_csv["sentimen"] = train_csv["sentiment"].map(label_mapping)

In [76]:
from torch.utils.data import Dataset, DataLoader

class SentimentDataset(Dataset):
    def __init__(self, embeddings, labels):
        self.embeddings = embeddings
        self.labels = labels

    def __len__(self):
        return len(self.embeddings)

    def __getitem__(self, idx):
        return self.embeddings[idx], self.labels[idx]

# Prepare dataset
embeddings = torch.stack(train_csv["sentence_embedding"].tolist())  # Shape: [num_samples, 768]
labels = torch.tensor(train_csv["sentimen"].tolist())  # Shape: [num_samples]

dataset = SentimentDataset(embeddings, labels)
train_loader = DataLoader(dataset, batch_size=16, shuffle=True)

In [77]:
import torch.nn as nn

class SentimentClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(SentimentClassifier, self).__init__()
        self.fc = nn.Linear(input_dim, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        return self.softmax(self.fc(x))


In [81]:
# Training loop
epochs = 100
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch_embeddings, batch_labels in train_loader:
        optimizer.zero_grad()

        # Ensure batch_labels are of type LongTensor
        batch_labels = batch_labels.long()
        
        outputs = model(batch_embeddings)
        loss = criterion(outputs, batch_labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(f"Epoch {epoch + 1}, Loss: {total_loss:.4f}")

Epoch 1, Loss: 29.0489
Epoch 2, Loss: 29.0405
Epoch 3, Loss: 29.0319
Epoch 4, Loss: 28.6610
Epoch 5, Loss: 28.4721
Epoch 6, Loss: 28.6467
Epoch 7, Loss: 29.1902
Epoch 8, Loss: 28.8197
Epoch 9, Loss: 28.6333
Epoch 10, Loss: 28.6300
Epoch 11, Loss: 28.6260
Epoch 12, Loss: 28.6238
Epoch 13, Loss: 28.8057
Epoch 14, Loss: 28.6181
Epoch 15, Loss: 28.9848
Epoch 16, Loss: 28.6142
Epoch 17, Loss: 28.6124
Epoch 18, Loss: 28.6113
Epoch 19, Loss: 28.9793
Epoch 20, Loss: 28.4230
Epoch 21, Loss: 28.6068
Epoch 22, Loss: 28.4202
Epoch 23, Loss: 28.6044
Epoch 24, Loss: 28.6034
Epoch 25, Loss: 28.6029
Epoch 26, Loss: 28.4159
Epoch 27, Loss: 28.6008
Epoch 28, Loss: 28.4142
Epoch 29, Loss: 28.5993
Epoch 30, Loss: 28.5989
Epoch 31, Loss: 28.5980
Epoch 32, Loss: 28.4113
Epoch 33, Loss: 28.7830
Epoch 34, Loss: 28.7826
Epoch 35, Loss: 28.4097
Epoch 36, Loss: 28.5958
Epoch 37, Loss: 28.9680
Epoch 38, Loss: 28.7813
Epoch 39, Loss: 28.4079
Epoch 40, Loss: 28.5942
Epoch 41, Loss: 28.4071
Epoch 42, Loss: 29.1532
E

In [80]:
from sklearn.metrics import accuracy_score

model.eval()
predictions, actuals = [], []
with torch.no_grad():
    for batch_embeddings, batch_labels in train_loader:  # Use test_loader for evaluation
        outputs = model(batch_embeddings)
        preds = torch.argmax(outputs, dim=1)
        predictions.extend(preds.tolist())
        actuals.extend(batch_labels.tolist())

accuracy = accuracy_score(actuals, predictions)
print(f"Accuracy: {accuracy:.4f}")


/Users/siddharth/Documents/US-Election-Sentiment/lib/python3.12/site-packages/sklearn/utils/_array_api.py:392: RuntimeWarning: invalid value encountered in cast
  return x.astype(dtype, copy=copy, casting=casting)


ValueError: Input y_true contains NaN.

In [96]:
%pip install setuptools


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [97]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

ModuleNotFoundError: No module named 'distutils'